<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignment 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- __[X]__ The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- __[X]__ Normalize the data (all features should have roughly the same scale)
- __[X]__ Import the type of model and layers that you will need from Keras.
- __[X]__ Instantiate a model object and use `model.add()` to add layers to your model
- __[X]__ Since this is a regression model you will have a single output node in the final layer.
- __[X]__ Use activation functions that are appropriate for this task
- __[X]__ Compile your model
- __[X]__ Fit your model and report its accuracy in terms of Mean Squared Error
- __[X]__ Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- __[X]__ Run this same data through a linear regression model. Which achieves higher accuracy?
- __[X]__ Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- __[X]__ After feature engineering, which model sees a greater accuracy boost due to the new features?

In [1]:
# Imports
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline

In [2]:
# Download boston housing data
tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

((array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
          3.96900e+02, 1.87200e+01],
         [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
          3.95380e+02, 3.11000e+00],
         [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
          3.75520e+02, 3.26000e+00],
         ...,
         [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
          3.62250e+02, 7.83000e+00],
         [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
          2.61950e+02, 1.57900e+01],
         [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
          3.76700e+02, 4.38000e+00]]),
  array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
         17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
         32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
         23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
         12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.

In [3]:
# split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

In [4]:
# Convert x and y trains and tests to dataframes with nice feature labels
x_train = pd.DataFrame(x_train, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
y_train = pd.DataFrame(y_train, columns=['MEDV'])
x_test = pd.DataFrame(x_test, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
y_test = pd.DataFrame(y_test, columns=['MEDV'])
x_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [5]:
#Check shapes and target df
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
y_train

(404, 13)
(404, 1)
(102, 13)
(102, 1)


,MEDV
0,15.2
1,42.3
2,50.0
3,21.1
4,17.7
...,...
399,19.4
400,25.2
401,19.4
402,19.4


In [6]:
# Normalize data
pipeline = Pipeline([('normalizer', Normalizer())])
x_train_norm = pipeline.fit_transform(x_train)
y_train_norm = pipeline.transform(y_train)
x_test_norm = pipeline.fit_transform(x_test)
y_test_norm = pipeline.transform(y_test)

In [7]:
# Load stuff for tensorboard
%load_ext tensorboard

import os
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [8]:
# Create TF/NN model, compile, and fit
model = Sequential([
    Dense(128, activation='relu', input_dim=13),
    Dense(1, activation="relu")
])

model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

model.fit(x=x_train_norm, 
        y=y_train_norm, 
        epochs=6, 
        validation_data=(x_test_norm, y_test_norm), 
        callbacks=[tensorboard_callback])

Train on 404 samples, validate on 102 samples
Epoch 1/6
404/404 [==============================] - 1s 2ms/sample - loss: 1.0000 - mae: 1.0000 - mse: 1.0000 - val_loss: 1.0000 - val_mae: 1.0000 - val_mse: 1.0000
Epoch 2/6
404/404 [==============================] - 0s 143us/sample - loss: 1.0000 - mae: 1.0000 - mse: 1.0000 - val_loss: 1.0000 - val_mae: 1.0000 - val_mse: 1.0000
Epoch 3/6
404/404 [==============================] - 0s 133us/sample - loss: 1.0000 - mae: 1.0000 - mse: 1.0000 - val_loss: 1.0000 - val_mae: 1.0000 - val_mse: 1.0000
Epoch 4/6
404/404 [==============================] - 0s 167us/sample - loss: 1.0000 - mae: 1.0000 - mse: 1.0000 - val_loss: 1.0000 - val_mae: 1.0000 - val_mse: 1.0000
Epoch 5/6
404/404 [==============================] - 0s 180us/sample - loss: 1.0000 - mae: 1.0000 - mse: 1.0000 - val_loss: 1.0000 - val_mae: 1.0000 - val_mse: 1.0000
Epoch 6/6
404/404 [==============================] - 0s 198us/sample - loss: 1.0000 - mae: 1.0000 - mse: 1.0000 - val_los

In [9]:
# Launch tensorboard logs

%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 25126), started 0:38:58 ago. (Use '!kill 25126' to kill it.)

In [10]:
# Linear regression comparison
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(x_train_norm, y_train_norm)

LinearRegression()

In [11]:
# Calculate MSE for 
from sklearn.metrics import mean_squared_error
y_pred = linear.predict(x_test_norm)
mean_squared_error(y_test_norm, y_pred)

0.0

In [12]:
# Launch tensorboard logs

%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 25126), started 0:38:58 ago. (Use '!kill 25126' to kill it.)

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- The number of nodes in your output layer should equal the number of classes you want to predict for Fashion-MNIST.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [13]:
# split between train and test sets
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [14]:
# X Variable Types
num_classes = 10

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Correct Encoding on Y
# What softmax expects = [0,0,0,0,0,1,0,0,0,0]
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [15]:
# Build model with sequential api
from tensorflow.keras.layers import Flatten

model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(10, activation="softmax"))

In [16]:
# Compile model using accuracy and adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Fit model using method 1
model.fit(X_train, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.4643 - accuracy: 0.8780s - loss: 0
Epoch 2/10
60000/60000 [==============================] - 4s 71us/sample - loss: 0.3033 - accuracy: 0.9155
Epoch 3/10
60000/60000 [==============================] - 3s 57us/sample - loss: 0.2828 - accuracy: 0.9213
Epoch 4/10
60000/60000 [==============================] - 4s 64us/sample - loss: 0.2730 - accuracy: 0.9233
Epoch 5/10
60000/60000 [==============================] - 4s 65us/sample - loss: 0.2664 - accuracy: 0.9258
Epoch 6/10
60000/60000 [==============================] - 4s 60us/sample - loss: 0.2618 - accuracy: 0.9275
Epoch 7/10
60000/60000 [==============================] - 3s 57us/sample - loss: 0.2579 - accuracy: 0.9286
Epoch 8/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.2554 - accuracy: 0.9293
Epoch 9/10
60000/60000 [==============================] - 4s 66us/sample - loss: 0.2530 - accuracy: 0.9296
Epo

In [18]:
# Fit model using validation data
model.fit(X_train, y_train, 
          epochs=5, 
          validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 67us/sample - loss: 0.2491 - accuracy: 0.9316 - val_loss: 0.2647 - val_accuracy: 0.9268
Epoch 2/5
60000/60000 [==============================] - 4s 66us/sample - loss: 0.2475 - accuracy: 0.9318 - val_loss: 0.2659 - val_accuracy: 0.9267
Epoch 3/5
60000/60000 [==============================] - 4s 61us/sample - loss: 0.2464 - accuracy: 0.9322 - val_loss: 0.2662 - val_accuracy: 0.9272
Epoch 4/5
60000/60000 [==============================] - 4s 62us/sample - loss: 0.2452 - accuracy: 0.9325 - val_loss: 0.2630 - val_accuracy: 0.9271
Epoch 5/5
60000/60000 [==============================] - 4s 65us/sample - loss: 0.2438 - accuracy: 0.9330 - val_loss: 0.2647 - val_accuracy: 0.9295


## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.